In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/AI_Dataset/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, Normalize, MotionBlur, Cutout
)
from keras.callbacks import ReduceLROnPlateau

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        if self.augmentation is None:
            image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        else:
            image = np.stack([self.augmentation(image=cv2.imread(item["image_path"]))["image"] for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
AUGMENTATIONS_TRAIN = Compose([
    HorizontalFlip(p=0.5),
    RandomContrast(limit=[0, 0.5], p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=[0, 0.5], p=0.5),
    CLAHE(p=1.0, clip_limit=2.0),
    #ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=0, p=0.8), 
    #ToFloat(max_value=255)
    #Cutout(p=1, num_holes=3, max_h_size=40, max_w_size=30)
    #MotionBlur(blur_limit=3,p=1)
    #Normalize()
])
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=AUGMENTATIONS_TRAIN)
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False, augmentation=None)

In [0]:
cv2_imshow(train_gen[111][0][0])
cv2_imshow(train_gen[103][0][0])

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

**VGG16 Definition**

In [0]:
from keras.applications import VGG16

backbone = VGG16(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


**Model Initialization**

In [0]:
from keras.optimizers import Adam

#opt = SGD(lr=0.001, momentum=0.9)
opt_nesterov = SGD(lr=0.003, momentum=0.9, nesterov=True)
opt_sgd = SGD(lr=0.003, momentum=0.9)
opt_adam = Adam(lr=0.003)

model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

losses = {
  "gender_output": "binary_crossentropy",
  "image_quality_output": "categorical_crossentropy",
  "age_output": "categorical_crossentropy",
  "weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy",

}
loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 10.0}

model.compile(loss=losses,
              loss_weights=loss_weights,
              optimizer=opt_nesterov,
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
reduce_gender_lr = ReduceLROnPlateau(monitor='val_gender_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_img_quality_lr = ReduceLROnPlateau(monitor='val_image_quality_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_age_lr = ReduceLROnPlateau(monitor='val_age_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_weight_lr = ReduceLROnPlateau(monitor='val_weight_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_bag_lr = ReduceLROnPlateau(monitor='val_bag_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_footwear_lr = ReduceLROnPlateau(monitor='val_footwear_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_pose_lr = ReduceLROnPlateau(monitor='val_pose_output_acc', factor=0.2, patience=3, min_lr=0.001)
reduce_emotion_lr = ReduceLROnPlateau(monitor='val_emotion_output_acc', factor=0.2, patience=3, min_lr=0.001)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=30,
    verbose=1,
    class_weight={0: 1., 1: 5., 2: 50., 3: 50., 4: 10., 5: 2., 6: 2., 7: 50.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr]
)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=30,
    verbose=1
)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=30,
    verbose=1
)